In [ ]:
#!/home/akhanal1/Spring2018/pl-env/bin/python3.5
import os
os.chdir('C:\\Projects\\ature')
import sys
import numpy as np
import itertools as itr
from random import shuffle
from commons.segmentation import AtureTest
from PIL import Image as IMG
from commons.IMAGE import Image, GlaucomaImage
from commons.accumulator import Accumulator
import cv2
import preprocess.utils.filter_utils as filutils


SK_THRESHOLD_PARAMS = np.arange(40, 61, 20)
ALPHA_PARAMS = np.arange(5, 7, 0.5)
GABOR_CONTRIBUTION_PARAMS = np.arange(0.6, 1.1, 0.2)
SEGMENTATION_THRESHOLD_PARAMS = np.arange(9, 15, 0.5)

PARAMS_ITR = itr.product(SK_THRESHOLD_PARAMS, ALPHA_PARAMS, GABOR_CONTRIBUTION_PARAMS, SEGMENTATION_THRESHOLD_PARAMS)

PARAMS_COMBINATION = list(PARAMS_ITR)
shuffle(PARAMS_COMBINATION)

keys = ('sk_threshold', 'alpha', 'gabor_contrib', 'seg_threshold')

all_params = list(dict(zip(keys, param)) for param in PARAMS_COMBINATION)

params = {'sk_threshold': 60,
          'alpha': 6.5,
          'gabor_contrib': 0.8,
          'seg_threshold': 9}

In [ ]:
### HD images
data_file_path = 'C:\\Projects\\ature\\data\\hrf\\glaucoma'
mask_path = 'C:\\Projects\\ature\\data\\hrf\\glaucoma_fovmask'
ground_truth_path = 'C:\\Projects\\ature\\data\\hrf\\glaucoma_manualsegm'
mask_suffix =  '_g_mask.tif'
ground_truth_suffix = '_g.tif'
input_image = '01_g.jpg'

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
def get_mask_file(file_name): return file_name.split('_')[0] + mask_suffix
def get_ground_truth_file(file_name): return file_name.split('_')[0] + ground_truth_suffix

img_obj = GlaucomaImage()

img_obj.load_file(data_dir=data_file_path, file_name=input_image)
img_obj.load_mask(mask_dir=mask_path, fget_mask=get_mask_file, erode=True)
img_obj.load_ground_truth(gt_dir=ground_truth_path, fget_ground_truth=get_ground_truth_file)

img_obj.working_arr = img_obj.image_arr[:, :, 1]
img_obj.apply_mask()

img_obj.apply_bilateral()
img_obj.apply_gabor()
img_obj.generate_lattice_graph(eight_connected=False)

accumulator = Accumulator(img_obj=img_obj)

In [ ]:
### Run for image files with in-time mask erosion
tester = AtureTest(out_dir='out')
acc = tester.run(params=params, save_images=False, epochs=1, alpha_decay=0.4, accumulator=accumulator)
# tester.run_all(params_combination=all_params)

In [ ]:
### Run for all images in a directory
# tester = AtureTest(data_dir=data_file_path, out_dir=os.path.join('out', 'out_new'))
# tester.run_all(params_combination=all_params)

In [ ]:
IMG.fromarray(img_obj.image_arr)

In [ ]:
IMG.fromarray(img_obj.diff_bilateral)

In [ ]:
IMG.fromarray(img_obj.img_gabor)

In [ ]:
IMG.fromarray(255-acc.res['skeleton0'])

In [ ]:
IMG.fromarray(acc.res['segmented0'])

In [ ]:
print(acc.res['scores0'])

In [ ]:
print(acc.res['params0'])

In [ ]:
IMG.fromarray(acc.res['segmented_rgb0'])

In [ ]:
import preprocess.utils.img_utils as ig

In [ ]:
ig.histogram(acc.img_obj.img_gabor, bins=30)

In [ ]:
fni = np.full(acc.arr_2d.shape, 250, dtype=np.uint8)
for i in range(fni.shape[0]):
    for j in range(fni.shape[1]):
        if acc.arr_2d[i,j] == 0 and acc.img_obj.ground_truth[i,j] == 255:
            fni[i, j] = acc.img_obj.img_gabor[i,j]

In [16]:
# IMG.fromarray(fni)
from abc import ABC, abstractmethod

In [17]:
class Ig(ABC):
    @abstractmethod
    def pss():
        pass
    

In [18]:
Ig

__main__.Ig

In [19]:
class IIg(Ig):
    pass

In [20]:
i = Ig()

TypeError: Can't instantiate abstract class Ig with abstract methods pss